# ScpTensor基础工作流程：质控与归一化

本notebook展示ScpTensor的完整质控(QC)和归一化工作流程，从数据加载到归一化处理。

**预计时间**: 30分钟  
**涵盖内容**:
- PSM层级质控（污染物过滤、PIF过滤）
- 样本层级质控（低质量样本过滤、双细胞检测）
- 特征层级质控（缺失率过滤、CV过滤）
- 归一化方法（对数、中位数、均值、分位数）

---

**作者**: ScpTensor Team  
**版本**: v0.1.0-beta  
**更新**: 2026-01-21


In [ ]:
# 导入必要的库
import matplotlib.pyplot as plt

# 导入ScpTensor核心模块
# 导入归一化模块
from scptensor import (
    create_test_container,
    norm_log,
    norm_mean,
    norm_median,
    norm_quantile,
)

# 导入QC模块
from scptensor.qc import (
    calculate_feature_qc_metrics,
    calculate_sample_qc_metrics,
    filter_contaminants,
    filter_doublets_mad,
    filter_features_by_missingness,
    filter_low_quality_samples,
    filter_psms_by_pif,
)

# 设置图形样式
plt.style.use(["science", "no-latex"])
plt.rcParams["figure.dpi"] = 300
plt.rcParams["figure.figsize"] = (10, 6)

print("✓ 所有库导入成功")
print("✓ 图形样式已设置 (SciencePlots, 300 DPI)")

## 1. 数据生成与加载

在实际分析中，您会从MS数据加载PSM和蛋白表达数据。这里我们生成模拟数据来演示完整的工作流程。

**模拟数据特点**:
- PSM层级：包含污染物（角蛋白、胰蛋白酶）和低PIF值的肽段
- 样本层级：包含低质量样本（空孔）和双细胞
- 特征层级：包含高缺失率和高变异系数的蛋白
- 批次效应：包含3个批次的数据


In [ ]:
# 生成模拟的SCP数据
print("正在生成模拟数据...")

# 创建测试container
container = create_test_container(
    n_samples=100,  # 100个细胞
    n_features=500,  # 500个蛋白
    missing_rate=0.3,  # 30%缺失率
    batch_ids=["batch1", "batch2", "batch3"],  # 3个批次
    seed=42,
)

print("✓ 数据生成完成")
print(f"  - 样本数: {container.n_samples}")
print(f"  - 蛋白数: {container.assays['protein'].n_features}")
print(f"  - 批次: {container.obs['batch'].unique().to_list()}")

## 2. PSM层级质控

### 2.1 过滤污染物

**污染物类型**:
- 角蛋白 (Keratin, KRT): 实验操作污染
- 胰蛋白酶: 消化酶残留
- 白蛋白/免疫球蛋白: 血液污染
- MaxQuant污染物: CON__, REV__

**方法**: `filter_contaminants()` 使用正则表达式匹配


In [ ]:
# 查看过滤前的PSM数量
n_psms_before = container.assays["peptide"].n_features
print(f"过滤前PSM数量: {n_psms_before}")

# 过滤污染物
container = filter_contaminants(
    container,
    assay_name="peptide",
    feature_col="gene_names",
)

n_psms_after = container.assays["peptide"].n_features
n_removed = n_psms_before - n_psms_after
print(f"过滤后PSM数量: {n_psms_after}")
print(f"移除的污染物: {n_removed} ({100 * n_removed / n_psms_before:.1f}%)")

### 2.2 PIF过滤（母离子纯度）

**PIF (Parent Ion Fraction)** 衡量前体离子分离的纯度

- **低PIF (<0.8)**: 共洗脱干扰，定量不可靠
- **推荐阈值**:
  - 0.8: 标准严格度（默认）
  - 0.9: 高严格度（关键实验）
  - 0.7: 低严格度（探索性研究）


In [ ]:
# 基于PIF过滤PSM
container = filter_psms_by_pif(
    container,
    assay_name="peptide",
    min_pif=0.8,  # 标准80%阈值
    pif_col="pif",
)

print("✓ PIF过滤完成")
print(f"  - 当前PSM数量: {container.assays['peptide'].n_features}")

## 3. 样本层级质控

### 3.1 计算样本QC指标

**计算指标**:
- `n_features_*`: 检测到的特征数（非零/非NaN）
- `total_intensity_*`: 总强度
- `log1p_total_intensity_*`: 对数转换后的总强度


In [ ]:
# 计算样本QC指标
container = calculate_sample_qc_metrics(
    container,
    assay_name="protein",
    layer_name="raw",
)

# 查看添加的指标
print("添加的QC指标:")
print([col for col in container.obs.columns if "protein" in col])

# 查看统计信息
print("\n特征数统计:")
print(container.obs["n_features_protein"].describe())

In [ ]:
# 可视化样本质量
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 左图：特征数分布
axes[0].hist(container.obs["n_features_protein"], bins=30, edgecolor="black", alpha=0.7)
axes[0].set_xlabel("检测到的蛋白数")
axes[0].set_ylabel("样本数")
axes[0].set_title("特征数分布")
axes[0].axvline(100, color="red", linestyle="--", label="最小阈值(100)")
axes[0].legend()

# 右图：总强度分布
axes[1].hist(container.obs["log1p_total_intensity_protein"], bins=30, edgecolor="black", alpha=0.7)
axes[1].set_xlabel("log(总强度 + 1)")
axes[1].set_ylabel("样本数")
axes[1].set_title("强度分布")

plt.tight_layout()
plt.show()

### 3.2 过滤低质量样本

**MAD (Median Absolute Deviation)** 方法检测离群值:
- 比固定阈值更鲁棒
- 适用于小样本 (<100)
- 可解释性强

**过滤标准**:
- `min_features=100`: 至少100个蛋白
- `nmads=3.0`: 3倍MAD之外的样本


In [ ]:
# 过滤低质量样本（Empty Wells）
n_samples_before = container.n_samples

container = filter_low_quality_samples(
    container,
    assay_name="protein",
    min_features=100,  # 最小特征数
    nmads=3.0,  # MAD阈值
    use_mad=True,  # 使用MAD而非固定阈值
)

n_samples_after = container.n_samples
print("✓ 低质量样本过滤完成")
print(f"  - 移除样本数: {n_samples_before - n_samples_after}")
print(f"  - 剩余样本数: {n_samples_after}")

### 3.3 过滤双细胞

**双细胞**: 两个细胞被误认为一个

**检测方法**: 基于总强度的右侧离群值
- 双细胞的总强度通常约为单细胞的2倍
- 使用MAD的`direction="upper"`参数


In [ ]:
# 基于总强度检测并过滤双细胞
n_before = container.n_samples

container = filter_doublets_mad(
    container,
    assay_name="protein",
    metric="total_intensity",
    nmads=3.0,
)

n_after = container.n_samples
print("✓ 双细胞过滤完成")
print(f"  - 检测到双细胞: {n_before - n_after}")
print(f"  - 剩余样本数: {n_after}")

## 4. 特征层级质控

### 4.1 计算特征QC指标

**计算指标**:
- `missing_rate`: 缺失率 (NaN或0的比例)
- `detection_rate`: 检测率 (1 - missing_rate)
- `mean_expression`: 平均表达量
- `cv`: 变异系数 (std/mean)


In [ ]:
# 计算特征QC指标
container = calculate_feature_qc_metrics(
    container,
    assay_name="protein",
    layer_name="raw",
)

# 查看添加的指标
print("添加的QC指标:")
print(
    [
        col
        for col in container.assays["protein"].var.columns
        if col in ["missing_rate", "detection_rate", "mean_expression", "cv"]
    ]
)

# 查看统计信息
print("\n缺失率统计:")
print(container.assays["protein"].var["missing_rate"].describe())

In [ ]:
# 可视化特征质量
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 左图：缺失率分布
axes[0].hist(container.assays["protein"].var["missing_rate"], bins=30, edgecolor="black", alpha=0.7)
axes[0].set_xlabel("缺失率")
axes[0].set_ylabel("蛋白数")
axes[0].set_title("蛋白缺失率分布")
axes[0].axvline(0.5, color="red", linestyle="--", label="50%阈值")
axes[0].legend()

# 右图：CV分布
cv_data = container.assays["protein"].var["cv"].drop_nulls()
axes[1].hist(cv_data, bins=30, edgecolor="black", alpha=0.7)
axes[1].set_xlabel("变异系数 (CV)")
axes[1].set_ylabel("蛋白数")
axes[1].set_title("蛋白变异系数分布")
axes[1].axvline(0.5, color="red", linestyle="--", label="CV=0.5")
axes[1].legend()

plt.tight_layout()
plt.show()

### 4.2 过滤高缺失率蛋白

**推荐阈值**:
- `max_missing_rate=0.5`: 最多50%缺失（宽松）
- `max_missing_rate=0.3`: 最多30%缺失（严格）


In [ ]:
# 过滤高缺失率蛋白
n_features_before = container.assays["protein"].n_features

container = filter_features_by_missingness(
    container,
    assay_name="protein",
    layer_name="raw",
    max_missing_rate=0.5,  # 最多50%缺失
)

n_features_after = container.assays["protein"].n_features
print("✓ 高缺失率蛋白过滤完成")
print(f"  - 移除蛋白数: {n_features_before - n_features_after}")
print(f"  - 剩余蛋白数: {n_features_after}")

### 4.3 过滤高变异系数蛋白（可选）

**CV解释**:
- **< 0.3**: 稳定蛋白，适合定量
- **0.3 - 0.5**: 中等变异，可接受
- **0.5 - 1.0**: 高变异，需谨慎
- **> 1.0**: 极高变异，建议过滤

**注意**: log转换会降低CV，建议先进行log转换再计算CV


In [ ]:
# 过滤高变异系数蛋白（可选）
# 注意：这里先不做log转换，展示CV过滤功能
# 在实际分析中，通常先log转换再过滤

# container = filter_features_by_cv(
#     container,
#     assay_name="protein",
#     layer_name="raw",
#     max_cv=0.5,
# )

print("提示：CV过滤在log转换后进行更合适")
print("我们将在归一化后演示CV过滤")

## 5. 归一化

归一化的目的是消除技术变异，使样本可比。

**常用归一化方法**:
1. **对数归一化** (`norm_log`): 稳定方差，处理非线性
2. **中位数归一化** (`norm_median`): 校正上样量差异
3. **均值归一化** (`norm_mean`): 类似中位数，但对异常值敏感
4. **分位数归一化** (`norm_quantile`): 强制所有样本具有相同分布


### 5.1 对数归一化

**推荐**: 在其他分析前先进行log转换

**数学公式**: `X_log = log(X + offset, base)`

**推荐参数**:
- `base=2.0`: log2转换（常用）
- `offset=1.0`: 避免log(0)


In [ ]:
# 对数归一化（强烈推荐）
container = norm_log(
    container,
    assay_name="protein",
    source_layer="raw",
    new_layer_name="log",
    base=2.0,  # log2转换
    offset=1.0,  # 避免log(0)
)

print("✓ 对数归一化完成")
print("  - 新layer: 'log'")
print("  - 数据范围: log2转换")

### 5.2 中位数归一化

**两种模式**:

1. **中心化模式** (`add_global_median=False`):
   - 每个样本减去其中位数
   - 结果：每个样本中位数 ≈ 0
   - 用途：消除样本间上样量差异

2. **缩放模式** (`add_global_median=True`):
   - 每个样本减去其中位数，然后加上全局中位数
   - 结果：所有样本对齐到全局中位数
   - 用途：保持整体数据尺度


In [ ]:
# 中位数归一化 - 两种模式对比

# 1. 中心化模式（推荐）
container = norm_median(
    container,
    assay_name="protein",
    source_layer="log",
    new_layer_name="log_centered",
    add_global_median=False,  # 中心化到0
)

print("✓ 中位数中心化完成")
print("  - 每个样本中位数 ≈ 0")

# 2. 缩放模式（可选）
container = norm_median(
    container,
    assay_name="protein",
    source_layer="log",
    new_layer_name="log_scaled",
    add_global_median=True,  # 对齐到全局中位数
)

print("✓ 中位数缩放完成")
print("  - 所有样本对齐到全局中位数")

### 5.3 均值归一化（可选）

**与中位数归一化类似，但使用均值而非中位数**

**注意**: 均值对异常值更敏感
- 如果数据有极端异常值，推荐使用中位数归一化
- 如果数据分布相对均匀，可以使用均值归一化


In [ ]:
# 均值归一化（可选）
container = norm_mean(
    container,
    assay_name="protein",
    source_layer="log",
    new_layer_name="log_mean_centered",
    add_global_mean=False,  # 中心化模式
)

print("✓ 均值归一化完成")
print("  - 每个样本均值 ≈ 0")

### 5.4 分位数归一化（高级）

**强制所有样本具有相同的分布**

**数学原理**:
1. 对每列（样本）排序
2. 计算行均值（参考分布）
3. 根据原始排名映射回参考分布

**用途**:
- 跨样本比较（热图、聚类）
- 数据需要严格对齐

**注意**: 必须先log转换


In [ ]:
# 分位数归一化
container = norm_quantile(
    container,
    assay_name="protein",
    source_layer="log",  # 使用log转换后的数据
    new_layer_name="log_quantile",
)

print("✓ 分位数归一化完成")
print("  - 所有样本具有相同的分布")

## 6. 结果对比与可视化

让我们对比不同归一化方法的效果...


In [ ]:
# 可视化不同归一化方法的效果
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 获取数据（只显示前10个样本）
n_samples_show = min(10, container.n_samples)

# 原始数据（log转换后）
X_log = container.assays["protein"].layers["log"].X[:n_samples_show, :]
axes[0, 0].boxplot([X_log[i, :] for i in range(n_samples_show)])
axes[0, 0].set_title("原始数据 (log转换)", fontsize=12, fontweight="bold")
axes[0, 0].set_ylabel("log2强度", fontsize=10)
axes[0, 0].set_xticklabels([])

# 中位数中心化
X_centered = container.assays["protein"].layers["log_centered"].X[:n_samples_show, :]
axes[0, 1].boxplot([X_centered[i, :] for i in range(n_samples_show)])
axes[0, 1].set_title("中位数中心化", fontsize=12, fontweight="bold")
axes[0, 1].set_ylabel("中心化强度", fontsize=10)
axes[0, 1].axhline(0, color="red", linestyle="--", linewidth=1)
axes[0, 1].set_xticklabels([])

# 中位数缩放
X_scaled = container.assays["protein"].layers["log_scaled"].X[:n_samples_show, :]
axes[1, 0].boxplot([X_scaled[i, :] for i in range(n_samples_show)])
axes[1, 0].set_title("中位数缩放", fontsize=12, fontweight="bold")
axes[1, 0].set_ylabel("缩放强度", fontsize=10)
axes[1, 0].set_xticklabels([])

# 分位数归一化
X_quantile = container.assays["protein"].layers["log_quantile"].X[:n_samples_show, :]
axes[1, 1].boxplot([X_quantile[i, :] for i in range(n_samples_show)])
axes[1, 1].set_title("分位数归一化", fontsize=12, fontweight="bold")
axes[1, 1].set_ylabel("归一化强度", fontsize=10)
axes[1, 1].set_xticklabels([])

plt.suptitle("不同归一化方法对比", fontsize=14, fontweight="bold", y=0.995)
plt.tight_layout()
plt.show()

## 7. 总结与建议

### 7.1 QC流程总结

✓ **PSM层级质控**:
- 过滤污染物（角蛋白、胰蛋白酶等）
- PIF过滤（推荐min_pif=0.8）

✓ **样本层级质控**:
- 计算QC指标（特征数、总强度）
- 过滤低质量样本（Empty Wells）
- 过滤双细胞（基于总强度右侧离群值）

✓ **特征层级质控**:
- 计算QC指标（缺失率、CV）
- 过滤高缺失率蛋白（max_missing_rate=0.5）
- 过滤高变异系数蛋白（max_cv=0.5，可选）

### 7.2 归一化方法选择

**推荐工作流程**:

1. **对数归一化** (必须):
   ```python
   container = norm_log(container, base=2.0, offset=1.0)
   ```
   - 稳定方差
   - 使数据更接近正态分布
   - 降低CV值

2. **中位数归一化** (推荐):
   ```python
   # 中心化模式（常用）
   container = norm_median(container, add_global_median=False)
   ```
   - 对异常值鲁棒
   - 消除上样量差异
   - 适合大多数SCP分析

3. **分位数归一化** (高级):
   ```python
   container = norm_quantile(container)
   ```
   - 强制所有样本具有相同分布
   - 适合跨样本比较
   - 适用于热图、聚类等可视化

### 7.3 下一步分析

完成QC和归一化后，可以进行:

1. **填补** (Imputation): 填充缺失值
   - KNN填补、MissForest等
   - 参见 `docs/tutorials/tutorial_03_imputation_integration.ipynb`

2. **批次校正** (Batch Correction): 消除批次效应
   - ComBat, Harmony等
   - 参见 `docs/tutorials/tutorial_03_imputation_integration.ipynb`

3. **降维** (Dimensionality Reduction): PCA, UMAP
   - 参见 `docs/tutorials/tutorial_04_clustering_visualization.ipynb`

4. **聚类** (Clustering): K-means, 图聚类
   - 参见 `docs/tutorials/tutorial_04_clustering_visualization.ipynb`

5. **差异表达分析** (Differential Expression)
   - 参见 `docs/tutorials/tutorial_05_differential_expression.ipynb`

### 7.4 参考文档

- **API文档**: [API Reference](../api/)
- **设计文档**: [Design Docs](../design/)
- **问题追踪**: [ISSUES_AND_LIMITATIONS.md](../ISSUES_AND_LIMITATIONS.md)

---

**恭喜！** 您已完成ScpTensor的基础工作流程学习。 🎉

现在您可以继续学习更高级的主题，或者开始分析您自己的数据！
